# QCFractal - NEB service

This notebook shows how an NEB service can be submmited and its record can be queried.

In [ ]:
from qcelemental.models import Molecule
import qcportal as ptl
from qcportal.neb import NEBKeywords
from qcelemental import constants
import json

A simple function to load a list of molecule objects from a json file.

In [ ]:
def load_neb_input(fname):
    chain = []
    schema_json = json.load(open(fname))
    for i, schema in schema_json.items():
        chain.append(Molecule.from_data(schema))
    return chain

## Creating a client object and preparing an input chain
After the client obejct is ready, we are preparing an input chain for the NEB service as well. 

In [ ]:
# Guest access
client = ptl.PortalClient("https://qcademo.molssi.org")

# Uncomment the following two lines to query a NEB record with a guest client.
#r = client.get_nebs([963])[0]
#print(r.stdout)

# If you have a username/password
#client = ptl.PortalClient("https://qcademo.molssi.org", username="YOUR_USERNAME", password="YOUR_PASSWORD")

chain = load_neb_input("NEB_input.json")

## Specifications

Calculation details can be adjusted here. 

### NEB keywords

* **images** - Number of images that will be used to locate a rough transition state structure. (default = 11)

* **spring_constant** - Spring constant in kcal/mol/Ang^2. (default = 1.0)

* **spring_type** - 0: Nudged Elastic Band (parallel spring force + perpendicular gradients) (default) \
                  1: Hybrid Elastic Band (full spring force + perpendicular gradients)\
                  2: Plain Elastic Band (full spring force + full gradients
                  
* **maximum_force** - Convergence criteria. Converge when maximum RMS-gradient (ev/Ang) of the chain fall below maximum_force. (default = 0.05)

* **average_force** - Convergence criteria. Converge when average RMS-gradient (ev/Ang) of the chain fall below average_force. (default = 0.025)

* **maximum_cycle** - Maximum iteration number for NEB calculation (default = 100)

* **energy_weighted** - Provide an integer value to vary the spring constant based on images' energy (range: spring_constant/energy_weighted - spring_constant) (default = None)

* **optimize_ts** - Setting it equal to true will perform a transition sate optimization starting with the guessed transition state structure from the NEB calculation result. (defualt = False)

* **optimize_endpoints** - Setting it equal to True will optimize two end points of the initial chain before starting NEB. (default = False)

* **coordinate_systm** - Coordinate system for optimizations:\
        "tric" for Translation-Rotation Internal Coordinates (default)\
        "cart" = Cartesian coordinate system\
        "prim" = Primitive (a.k.a redundant internal coordinates)\
        "dlc" = Delocalized Internal Coordinates,\
        "hdlc" = Hybrid Delocalized Internal Coordinates\
        "tric-p" for primitive Translation-Rotation Internal Coordinates (no delocalization)
        
* **epsilon** - Small eigenvalue threshold for resetting Hessian. (default = 1e-5)

* **hessian_reset** - Reset Hessian when eigenvalues are below the epsilon. If it is set to False, it will skip updating the hessian. (default = True)

In [ ]:
sp_spec = {
    'program':'psi4',
    'driver':'energy',
    'method':'b3lyp',
    'basis':'6-31g',
    'keywords':{},
}

kw = NEBKeywords(
    images= 11,
    spring_constant=0.1,
    optimize_ts=True,
    maximum_force= 0.05,
    average_force= 0.025,
    coordinate_system= 'tric',
    optimize_endpoints=True,
    spring_type=0)

meta, ids = client.add_nebs([chain], 'geometric', sp_spec, kw)

We can query the record and print stdout.

In [ ]:
r = client.get_nebs(ids)[0]
print(r.stdout)

Other useful records are..

In [ ]:
singlepoint_records = r.singlepoints # {int(iteration number) : [siglepoint records]}
guessed_TS_moelcule = r.neb_result # Molecule object
optimization_records = r.optimizations # {"initial":Optimization record of the first image, "final":Optimization record of the last image
                                       #  "transition":Optimization record of the neb_result}